In [ ]:
import pandas as pd
import utils 
import plotly.graph_objects as go 
import instrument

In [ ]:
pair = "CAD_CHF"
granularity = "H1"
ma_list=[16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [ ]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ["time", "volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [ ]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [ ]:
df_ma.head()

In [ ]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 
    return 0

In [ ]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [ ]:
df_trades.head()

In [ ]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [ ]:
df_trades["GAIN"].sum()

In [ ]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [ ]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)

In [ ]:
df_trades.info()

In [49]:
df_all = pd.read_pickle("all_trades.pkl")

In [50]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MALONG,MASHORT,DURATION
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.0,5.0,5.00000
mean,1.470004,0.000664,0.000326,0.200000,15.680000,-106.00000,4.0,8.0,4.80000
std,0.009111,0.001866,0.001028,1.095445,154.373871,100.46701,0.0,0.0,1.30384
min,1.456340,-0.001118,-0.000491,-1.000000,-172.000000,-250.20000,4.0,8.0,3.00000
25%,1.468140,-0.000750,-0.000065,-1.000000,-28.800000,-172.00000,4.0,8.0,4.00000
50%,1.470640,0.000428,-0.000063,1.000000,-25.000000,-54.00000,4.0,8.0,5.00000
75%,1.473540,0.001196,0.000131,1.000000,54.000000,-28.80000,4.0,8.0,6.00000
max,1.481360,0.003565,0.002119,1.000000,250.200000,-25.00000,4.0,8.0,6.00000


In [ ]:
df_plot = df_ma.iloc[:100].copy()

In [ ]:
df_plot.shape

In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()